# Model - All Routes

## Data Preparation and Preprocessing

In [1]:
import pandas as pd
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import warnings
from joblib import dump

In [2]:
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("../Data/All.csv")

In [4]:
df

route_id route_short_name  direction_id  stop_id  \
0         11638               21             0    10573   
1         11638               21             0    13844   
2         11638               21             0     2472   
3         11638               21             0     2473   
4         11638               21             0     2474   
...         ...              ...           ...      ...   
26152     11648               37             1     1433   
26153     11648               37             1     5323   
26154     11648               37             1     5326   
26155     11648               37             1     7093   
26156     11648               37             1     7094   

                                               stop_name  vehicle_id  \
0                         Washington St & Ashland Ave NB       13039   
1                         25th St & Greenmount Ave WB FS       13039   
2                            Washington St & Eager St NB       13039   
3                            Washington St & Chase St NB       13039   
4                          Washington St & Preston St NB       13039   
...                                                  ...         ...   
26152                    Liberty Rd & Marriotts Ln FS NB       16004   
26153                          Liberty Rd & Libertypl WB       16004   
26154  Liberty Rd & Liberty Crossroads Shopping Cente...       16004   
26155                Milford Mill Rd & Liberty Rd Mid NB       16004   
26156                 Liberty Rd & Milford Mill Rd FS NB       16004   

       driver_id start_time  sched_adherence_secs           scheduled_date  \
0         1980.0      11:10               842.555  2023-10-01 00:00:00.000   
1         1980.0      11:10               695.070  2023-10-01 00:00:00.000   
2         1980.0      11:10               821.367  2023-10-01 00:00:00.000   
3         1980.0      11:10               820.224  2023-10-01 00:00:00.000   
4         1980.0      11:10               832.819  2023-10-01 00:00:00.000   
...          ...        ...                   ...                      ...   
26152     2028.0      13:04                61.581  2023-10-31 00:00:00.000   
26153     2028.0      13:04               102.328  2023-10-31 00:00:00.000   
26154     2028.0      13:04                82.252  2023-10-31 00:00:00.000   
26155     2028.0      13:04                79.876  2023-10-31 00:00:00.000   
26156     2028.0      13:04                74.984  2023-10-31 00:00:00.000   

       ... matched_location Agency     incident_type Agency-specific Type  \
0      ...    WASHINGTON ST   Waze  Disabled Vehicle        Weatherhazard   
1      ...   GREENMOUNT AVE   Waze  Disabled Vehicle        Weatherhazard   
2      ...    WASHINGTON ST   Waze  Disabled Vehicle        Weatherhazard   
3      ...    WASHINGTON ST   Waze  Disabled Vehicle        Weatherhazard   
4      ...    WASHINGTON ST   Waze  Disabled Vehicle        Weatherhazard   
...    ...              ...    ...               ...                  ...   
26152  ...       LIBERTY RD   Waze     Animal Struck               Hazard   
26153  ...       LIBERTY RD   Waze     Animal Struck               Hazard   
26154  ...       LIBERTY RD   Waze     Animal Struck               Hazard   
26155  ...       LIBERTY RD   Waze     Animal Struck               Hazard   
26156  ...       LIBERTY RD   Waze     Animal Struck               Hazard   

         Agency-specific Sub Type                 Start time  \
0      Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   
1      Hazard On Road Car Stopped  2023-10-02 11:41:59+00:00   
2      Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   
3      Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   
4      Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   
...                           ...                        ...   
26152    Hazard On Road Road Kill  2023-11-01 13:10:46+00:00   
26153    Hazard On Road Road Kill  2023-11-01 13:10:46+00:0

In [5]:
# mapping route_id to route name
route_mapping = pd.Series(df['route_id'].values, index=df['route_short_name']).to_dict()

In [6]:
# Extracting day and month from date
df['actual_time'] = pd.to_datetime(df['actual_time'])
df['day'] = df['actual_time'].dt.dayofweek
days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['day'] = df['day'].map(days)
df['month'] = df['actual_time'].dt.month_name()


In [7]:
# choosing columns from data to use for the model
data = df[['route_id', 'vehicle_id', 'driver_id', 'direction_id', 'sched_adherence_secs', 'incident_type', 'Agency-specific Sub Type', 'location', 'day', 'month']]

In [8]:
data.rename(columns={'incident_type': 'incident_category', 'Agency-specific Sub Type': 'incident_type'}, inplace=True)

In [9]:
# Separating features and target
X = data.drop(['sched_adherence_secs'], axis=1)
y = data['sched_adherence_secs']

In [10]:
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
# Defining numerical and categorical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]

In [12]:
# One-hot encoding categorical columns
mapper = DataFrameMapper([
    (numerical_cols, StandardScaler()),
    (categorical_cols, OneHotEncoder(handle_unknown='ignore'))
], df_out=True)


## Model

### Random Forest

In [13]:
# Creating the model with a Random Forest Regressor
model = Pipeline(steps=[
    ('mapper', mapper),
    ('regressor', RandomForestRegressor())
])

In [14]:
# Fit the model
model.fit(X_train, y_train)
# Predict and evaluate the model (r^2 score)
model.score(X_test, y_test)

0.825928691794997

In [15]:
# Extracting the model and route_mapping dictionary to use with frontend
dump(route_mapping, '../Frontend/route_mapping.pkl')
dump(model, '../Frontend/model.pkl')

['../Frontend/model.pkl']

## Making sample predictions

The cells below are not part of the model. They are used to demonstrate how the model can be used to make predictions about the schedule adherence of a bus if there is an incident on the route. 

The data used for these sample predictions are real records from `All.csv` which can be found in the `All_test.csv`. These 5 records were randomly moved from `All.csv` to `All_test.csv` to demonstrate the use of the model. These 5 records are not a part of the training or testing data used earlier in the code to build and evaluate the model

In [16]:
# Classifying whether the bust was early, on-time, or late.
def classify_time(value):
    if value < -120:
        return 'early'
    elif value <= 420:
        return 'on-time'
    else:
        return 'late'

In [17]:
# Function to create a prediction
def predict(model, route_name, direction_id, vehicle_id, driver_id, incident_category, incident_type, location, day, month):
    route_id = route_mapping[route_name]
    
    sample = {
    'route_id': route_id,
    'direction_id': direction_id,
    'vehicle_id': vehicle_id,
    'driver_id': driver_id,
    'incident_category': incident_category,
    'incident_type': incident_type,
    'location': location,
    'day': day,
    'month': month
    }
    
    sample_df = pd.DataFrame([sample])
    return model.predict(sample_df)
    

### Prediction 1

In [18]:
route_name = '78'
direction_id = 0
vehicle_id = 11010
driver_id = 1901
incident_category = 'Disabled Vehicle'
incident_type = 'Hazard On Road Car Stopped'
location = 'BALTIMORE ST'
day = 'Monday'
month = 'October'

actual_adherence_secs = 356.369

In [19]:
predicted_adherence_secs = predict(model, str(route_name), int(direction_id), int(vehicle_id), int(driver_id), str(incident_category), str(incident_type), str(location), str(day), str(month))

print(f"Predicted Schedule Adherence Seconds: {predicted_adherence_secs[0]}")
print(f"Prediction: {classify_time(predicted_adherence_secs[0])}")
print()
print(f"Actual Schedule Adherence Seconds: {actual_adherence_secs}")
print(f"Actual: {classify_time(actual_adherence_secs)}")

Predicted Schedule Adherence Seconds: 379.1878478292264
Prediction: on-time

Actual Schedule Adherence Seconds: 356.369
Actual: on-time


### Prediction 2

In [20]:
route_name = '83'
direction_id = 0
vehicle_id = 11035
driver_id = 3207
incident_category = 'Disabled Vehicle'
incident_type = 'Hazard On Road Car Stopped'
location = 'REISTERSTOWN RD'
day = 'Thursday'
month = 'October'

actual_adherence_secs = 306.968

In [21]:
predicted_adherence_secs = predict(model, str(route_name), int(direction_id), int(vehicle_id), int(driver_id), str(incident_category), str(incident_type), str(location), str(day), str(month))

print(f"Predicted Schedule Adherence Seconds: {predicted_adherence_secs[0]}")
print(f"Prediction: {classify_time(predicted_adherence_secs[0])}")
print()
print(f"Actual Schedule Adherence Seconds: {actual_adherence_secs}")
print(f"Actual: {classify_time(actual_adherence_secs)}")

Predicted Schedule Adherence Seconds: 242.88813169968293
Prediction: on-time

Actual Schedule Adherence Seconds: 306.968
Actual: on-time


### Prediction 3

In [22]:
route_name = '56'
direction_id = 1
vehicle_id = 20058
driver_id = 3252
incident_category = 'Collision'
incident_type = 'Accident Minor'
location = 'PHILADELPHIA RD'
day = 'Tuesday'
month = 'October'

actual_adherence_secs = 1286.094

In [23]:
predicted_adherence_secs = predict(model, str(route_name), int(direction_id), int(vehicle_id), int(driver_id), str(incident_category), str(incident_type), str(location), str(day), str(month))

print(f"Predicted Schedule Adherence Seconds: {predicted_adherence_secs[0]}")
print(f"Prediction: {classify_time(predicted_adherence_secs[0])}")
print()
print(f"Actual Schedule Adherence Seconds: {actual_adherence_secs}")
print(f"Actual: {classify_time(actual_adherence_secs)}")

Predicted Schedule Adherence Seconds: 1347.4128420202105
Prediction: late

Actual Schedule Adherence Seconds: 1286.094
Actual: late


### Prediction 4

In [24]:
route_name = '70'
direction_id = 0
vehicle_id = 16027
driver_id = 2307
incident_category = 'Obstructions'
incident_type = 'Hazard On Road Object'
location = 'CRAIN HWY'
day = 'Tuesday'
month = 'October'

actual_adherence_secs = -92.846

In [25]:
predicted_adherence_secs = predict(model, str(route_name), int(direction_id), int(vehicle_id), int(driver_id), str(incident_category), str(incident_type), str(location), str(day), str(month))

print(f"Predicted Schedule Adherence Seconds: {predicted_adherence_secs[0]}")
print(f"Prediction: {classify_time(predicted_adherence_secs[0])}")
print()
print(f"Actual Schedule Adherence Seconds: {actual_adherence_secs}")
print(f"Actual: {classify_time(actual_adherence_secs)}")

Predicted Schedule Adherence Seconds: -35.132181200528
Prediction: on-time

Actual Schedule Adherence Seconds: -92.846
Actual: on-time


### Prediction 5

In [26]:
route_name = 'CityLink BROWN'
direction_id = 1
vehicle_id = 14019
driver_id = 3563
incident_category = 'Disabled Vehicle'
incident_type = 'Hazard On Road Car Stopped'
location = 'PRATT ST'
day = 'Wednesday'
month = 'October'

actual_adherence_secs = -6.732

In [27]:
predicted_adherence_secs = predict(model, str(route_name), int(direction_id), int(vehicle_id), int(driver_id), str(incident_category), str(incident_type), str(location), str(day), str(month))

print(f"Predicted Schedule Adherence Seconds: {predicted_adherence_secs[0]}")
print(f"Prediction: {classify_time(predicted_adherence_secs[0])}")
print()
print(f"Actual Schedule Adherence Seconds: {actual_adherence_secs}")
print(f"Actual: {classify_time(actual_adherence_secs)}")

Predicted Schedule Adherence Seconds: -46.28749180333557
Prediction: on-time

Actual Schedule Adherence Seconds: -6.732
Actual: on-time
